In [1]:
from facenet_pytorch import InceptionResnetV1, fixed_image_standardization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from PIL import Image
import random
import torch

from torchvision.datasets import ImageFolder
from torchvision import transforms
from torchvision import models as models
import torch.nn as nn
import torchvision
import torch.optim as optim
from torch.backends import cudnn
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn.metrics import accuracy_score
from scipy.stats import sem
from sklearn.metrics import confusion_matrix

from collections import defaultdict

import albumentations as A

In [3]:
# преобразовать в соответствии с логикой
test_images_path = 'test_dataset/'

def load_images_from_folder(folder): 
    images = []
    for filename in os.listdir(folder): # читаю все предобратанные картинки с фигурами
        print(os.path.join(folder,filename))
        images.append(os.path.join(folder,filename))
    return images

images = load_images_from_folder(test_images_path)
#images = pd.DataFrame(images)

test_dataset/4.png
test_dataset/5.png
test_dataset/7.jpg
test_dataset/6.png
test_dataset/2.png
test_dataset/3.png
test_dataset/1.png


In [7]:
genderModel = InceptionResnetV1(
    classify=True,
    pretrained='vggface2',
    num_classes= 2)

genderModel.load_state_dict(torch.load('model0.pth'))
genderModel.eval()

#ageModel = InceptionResnetV1(
#    classify=True,
#    pretrained='vggface2',
#    num_classes= 2)
#
#ageModel.load_state_dict(torch.load('model0.pth'))
#ageModel.eval()

trans = transforms.Compose([
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])

class GenderDataset(Dataset):
    def __init__(self, path, image_files, labels_age, labels_gender, p_augment=0.5,  validation=False):
        self.path = path
        self.X = image_files
        self.y_age = labels_age
        self.y_gender = labels_gender
        self.resize = A.Resize(160, 160, always_apply=True)
        self.transform = trans
        
    def __len__(self):
        return (len(self.X))
    
    def __getitem__(self, i):
        image = Image.open(self.path + self.X[i])
        image = image.convert('RGB')
        image = np.asarray(image)
        image = self.resize(image=image)['image']
        image = self.transform(image)
        label_age = self.y_age[i]
        label_gender = self.y_gender[i]
        
        return torch.tensor(image, dtype=torch.float), torch.tensor(label_gender, dtype=torch.long)

def predict_by_image(image_path):
    device = torch.device("cpu")
    results = torch.empty(0).to(device)

    dataset = GenderDataset('',
                            [image_path], 
                            [0],
                            [0], 
                            validation=True)

    loader = DataLoader(dataset, batch_size=1, shuffle=False)

    for batch in loader:
        batch_data, batch_label = batch
        predict = genderModel(batch_data)

        results = torch.cat((results,
                             nn.functional.softmax(predict.detach(),dim=1)), 0)

        results = results.cpu().numpy()
    return results

In [17]:
predict = predict_by_image(images[5])
print(images[5], predict)

test_dataset/3.png [[0.99742657 0.00257345]]


<ipython-input-7-ac2e1cac41bc>:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(image, dtype=torch.float), torch.tensor(label_gender, dtype=torch.long)
